# NER and Sentiment

In this section we will work through applying basic sentiment analysis to our data using a pre-built *distilBERT* model from the **Flair** library. We will then use our organization labels captured through NER in the previous section to create a list of organizations with the highest and lowest average sentiment scores.

In [1]:
import pandas as pd
import flair

We initialize the English sentiment model `en-sentiment`:

In [2]:
model = flair.models.TextClassifier.load('en-sentiment')

2022-03-01 17:06:36,275 loading file /Users/adenevreze/.flair/models/sentiment-en-mix-distillbert_4.pt


For each sample there are a few steps we need to take to create the sentiment score. We need to tokenize the input text, make a prediction, extract the direction (*positive* or *negative*) and confidence (a score from 0 to 1). If this is new to you, we cover the Flair sentiment model in more depth in **TK insert link**.

The following function carries out each of these steps for a single extract:

In [7]:
def get_sentiment(text):
    # tokenize input text
    sentence = flair.data.Sentence(str(text))
    # make sentiment prediction
    model.predict(sentence)
    # extract sentiment direction and confidence (label and score) object
    sentiment = sentence.labels[0]
    return sentiment

We now need to load our previously processed dataframe (which includes the *organizations* column) and `apply` the `get_sentiment` function to the *selftext* column. These sentiment scores will then be stored in a new *sentiment* column.

In [4]:
# load data
df = pd.read_csv('./data/processed/reddit_investing_ner.csv', sep='|')
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations
0,1.646144e+09,0.0,t3_t48xbu,1.0,My 9yr old wants to invest his 300$ so he can ...,investing,Investment options for children?,1.0,1.00,[]
1,1.646130e+09,0.0,t3_t44pro,597.0,"Again, the world's major chip and semiconducto...",investing,Ukraine supplies 70% of the world's neon. Chip...,597.0,0.95,[]
2,1.646129e+09,0.0,t3_t44j5g,2.0,Have a general question? Want to offer some c...,investing,Daily General Discussion and Advice Thread - M...,2.0,0.76,[]
3,1.646091e+09,0.0,t3_t3t1i6,15.0,Can someone explain how/why something like the...,investing,How would the RSX get Delisted?,15.0,0.77,"['ADR', 'E-Trade', 'RSX']"
4,1.646075e+09,0.0,t3_t3n3cz,0.0,Many years ago I had money put in DODBX for me...,investing,trying to figure DODBX out,0.0,0.14,[]


In [8]:
# get sentiment
df['sentiment'] = df['selftext'].apply(get_sentiment)
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations,sentiment
0,1.646144e+09,0.0,t3_t48xbu,1.0,My 9yr old wants to invest his 300$ so he can ...,investing,Investment options for children?,1.0,1.00,[],NEGATIVE (1.0)
1,1.646130e+09,0.0,t3_t44pro,597.0,"Again, the world's major chip and semiconducto...",investing,Ukraine supplies 70% of the world's neon. Chip...,597.0,0.95,[],NEGATIVE (0.996)
2,1.646129e+09,0.0,t3_t44j5g,2.0,Have a general question? Want to offer some c...,investing,Daily General Discussion and Advice Thread - M...,2.0,0.76,[],NEGATIVE (0.9993)
3,1.646091e+09,0.0,t3_t3t1i6,15.0,Can someone explain how/why something like the...,investing,How would the RSX get Delisted?,15.0,0.77,"['ADR', 'E-Trade', 'RSX']",NEGATIVE (1.0)
4,1.646075e+09,0.0,t3_t3n3cz,0.0,Many years ago I had money put in DODBX for me...,investing,trying to figure DODBX out,0.0,0.14,[],POSITIVE (0.9874)


Now we need to extract each of the organizations alongside it's sentiment score. We will then loop through each, tallying up a total sentiment score and count.

Before we do that, we need to convert each value in the *organizations* column to a list (they are currently strings because we cannot save Python lists to file within Pandas dataframes, they are automatically converted to strings).

In [9]:
import ast

df['organizations'] = df['organizations'].apply(lambda x: ast.literal_eval(x))

In [10]:
# initialize sentiment dictionary
sentiment = {}

# loop through dataframe and extract org labels and sentiment scores into sentiment dictionary
for i, row in df.iterrows():
    # extract sentiment direction and score
    direction = row['sentiment'].value
    score = row['sentiment'].score
    # loop through each label in organizations column
    for org in row['organizations']:
        # check if org label exists in sentiment dictionary already
        if org not in sentiment.keys():
            # if it doesn't, initialize new entry in dictionary
            sentiment[org] = {'POSITIVE': [], 'NEGATIVE': []}
        # append positive/negative score to respective dictionary entry
        sentiment[org][direction].append(score)

In [11]:
sentiment['ARK']

{'POSITIVE': [], 'NEGATIVE': [0.6586734652519226, 0.9972949624061584]}

Now we can loop through each organization entry in the sentiment dictionary and calculate an average positive, and average negative score:

In [12]:
# initialize sentiment list
avg_sentiment = []

# loop through each organization
for org in sentiment.keys():
    # get number of positive and negative ratings
    freq = len(sentiment[org]['POSITIVE']) + len(sentiment[org]['NEGATIVE'])
    for direction in ['POSITIVE', 'NEGATIVE']:
        # assign to variable for cleaner code
        score = sentiment[org][direction]
        # if there are no entries, set to 0
        if len(score) == 0:
            sentiment[org][direction] = 0.0
        else:
            # otherwise calculate total
            sentiment[org][direction] = sum(score)
    # now calculate total amount
    total = sentiment[org]['POSITIVE'] - sentiment[org]['NEGATIVE']
    # and the average score
    avg = total/freq
    # add to sentiment list
    avg_sentiment.append({
        'entity': org,
        'positive': sentiment[org]['POSITIVE'],
        'negative': sentiment[org]['NEGATIVE'],
        'frequency': freq,
        'score': avg
    })

In [13]:
sentiment_df = pd.DataFrame(avg_sentiment)
sentiment_df.head()

,entity,positive,negative,frequency,score
0,ADR,0.984114,1.997665,3,-0.337851
1,E-Trade,0.000000,0.999951,1,-0.999951
2,RSX,0.000000,0.999951,1,-0.999951
3,Gydan,0.000000,0.999728,1,-0.999728
4,The Board of Shell,0.000000,0.999728,1,-0.999728


Immediately we can see we have a lot of entities which have appeared once in our dataset, and because of this their score will be pushed to one extreme or the other. We can filter out anything with less than or equal to a frequency of `3` to remove many of these instances:

In [14]:
sentiment_df = sentiment_df[sentiment_df['frequency'] > 3]
sentiment_df

,entity,positive,negative,frequency,score
15,EPS,0.000000,14.876996,16,-0.929812
19,EU,0.954517,8.853503,10,-0.789899
27,SPY,0.645001,12.492278,14,-0.846234
48,Fed,1.397995,29.483101,32,-0.877660
50,FED,0.000000,7.906444,8,-0.988306
52,NASDAQ,0.977827,9.859250,11,-0.807402
57,QQQ,1.599753,3.977915,6,-0.396360
58,VTI,0.000000,5.166603,6,-0.861100
61,Fidelity,2.863389,15.738858,19,-0.677656
69,PE,0.000000,7.623611,8,-0.952951


Here we have some more relevant information. We can see a few items that we can remove through the `BLACKLIST` covered in earlier sections such as *Fed* and *Treasury*, but nonetheless this list is looking much better than before. We can apply `sort` to search for the entities with the highest overall score:

In [15]:
sentiment_df.sort_values('score', ascending=False).head(10)

,entity,positive,negative,frequency,score
181,Ford,1.891464,2.499279,5,-0.121563
183,Tesla,2.816353,4.629613,8,-0.226658
57,QQQ,1.599753,3.977915,6,-0.396360
97,Vanguard,2.794468,7.964769,12,-0.430858
389,Microsoft,3.479545,9.858380,14,-0.455631
91,ROTH,1.160482,3.920535,6,-0.460009
85,Apple,3.404489,10.645748,15,-0.482751
283,Visa,0.996748,2.944920,4,-0.487043
548,Interactive Brokers,0.942446,2.960019,4,-0.504393
235,AI,0.563329,2.618715,4,-0.513846


Very quickly we've got our results that we have pulled together using simple, ready-to-use models, and **zero** text preprocessing. With further fine-tuning, and process development, these already good results can become great. Which we will cover soon.